# 네이버에서 업종별 자치구별 식당 평점 크롤링하기

## 데이터 및 모듈 불러오기

In [1]:
# 필요한 모듈 불러오기
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm import tqdm
from urllib.parse import quote
from folium import *
from folium.plugins import MarkerCluster
import folium
import json

In [2]:
# 데이터 불러오기
store_df = pd.read_csv('./data_preprocessing/csv/Preprocessing_Seoul_store_data.csv')
seoul_df = pd.read_csv('D:/박상춘/프로젝트/raw data/서울시 행정 데이터/주민등록인구_20230214130825.csv', header=1)

## 리스트 생성

In [3]:
# 음식 업종 리스트
food_arr = store_df.food.unique()
food_lis = food_arr.tolist()
food_lis.remove('테스트')
food_lis.remove('심부름')
food_lis.remove('아시안/양식')
food_lis.remove('족발/보쌈')
food_lis.remove('돈까스/일식')
food_lis.extend(['아시안음식', '양식', '족발', '보쌈', '돈까스', '일식'])

In [5]:
food_lis

['치킨',
 '한식',
 '패스트푸드',
 '회',
 '분식',
 '도시락',
 '중식',
 '야식',
 '찜탕',
 '피자',
 '카페/디저트',
 '배달전문업체',
 '아시안음식',
 '양식',
 '족발',
 '보쌈',
 '돈까스',
 '일식']

In [4]:
# 자치구 리스트
gu_arr = seoul_df['동별(2)'].unique()
gu_lis = gu_arr.tolist()
gu_lis.remove('소계')

In [7]:
gu_lis

['종로구',
 '중구',
 '용산구',
 '성동구',
 '광진구',
 '동대문구',
 '중랑구',
 '성북구',
 '강북구',
 '도봉구',
 '노원구',
 '은평구',
 '서대문구',
 '마포구',
 '양천구',
 '강서구',
 '구로구',
 '금천구',
 '영등포구',
 '동작구',
 '관악구',
 '서초구',
 '강남구',
 '송파구',
 '강동구']

## 크롤링 테스트

In [8]:
# 드라이버로 링크 열기
url = 'https://map.naver.com/'

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
driver = webdriver.Chrome('chromedriver', options=options)

driver.get(url)
time.sleep(2)

In [9]:
# 자치구로 이동 (구로구)
driver.find_element(By.CSS_SELECTOR, '.input_search').send_keys(['구로구', Keys.ENTER])
time.sleep(1)

In [14]:
# 음식업종 검색 (치킨)
driver.find_element(By.CSS_SELECTOR, '.button_clear').click()
driver.find_element(By.CSS_SELECTOR, '.input_search').send_keys(['치킨', Keys.ENTER])
time.sleep(1)

In [15]:
# iframe 스위치
driver.switch_to.frame('searchIframe')

In [16]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [17]:
# 식당 리스트 가져오기
place_lis = soup.select('.Ryr1F > ul > li')

In [18]:
len(place_lis)

10

In [ ]:
# 식당 하나 지정
place = place_lis[0]
place

In [120]:
# 식당 이름 및 평점 가져오기
name = place.select_one('.place_bluelink').get_text()
try:
    rating = place.select_one('.a2RFq').get_text().strip('별점') # 식당 평점
except:
    rating = None # 평점이 없으면 Null 값으로 지정

In [121]:
print(name, rating)

치킨장인 4.56


In [123]:
# 주소 찾기
a = place.find('div').find('a')['class'][0]
driver.find_elements(By.CSS_SELECTOR, '.'+a)[0].click()
driver.switch_to.default_content()
driver.switch_to.frame('entryIframe')

In [124]:
soup1 = BeautifulSoup(driver.page_source, 'html.parser')

In [127]:
addr = soup1.select_one('.LDgIH').get_text()
addr

'서울 영등포구 선유동1로 14-1 1층'

In [ ]:
if addr.split()[0] == '서울':
    gu_temp = addr.split()[1].strip()
else:
    # continue

In [93]:
page_lis = len(soup.select('.zRM9F > a')) - 2
page_lis

5

In [33]:
# 다음페이지
driver.find_elements(By.CSS_SELECTOR, '.eUTV2')[1].click()

## 크롤링으로 평점 가져오기

### 자치구별 맛집 검색

In [27]:
lis = []

for gu in gu_lis: # 자치구별
    
    # 드라이버로 링크 열기
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")
    driver = webdriver.Chrome('chromedriver', options=options)

    driver.get('https://map.naver.com/')
    time.sleep(2)

    # driver.switch_to.default_content()
    driver.find_element(By.CSS_SELECTOR, '.input_search').send_keys([gu, Keys.ENTER]) # 자치구 검색
    print('===================================\n', gu)
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, '.button_clear').click() # 검색창 클리어
    driver.find_element(By.CSS_SELECTOR, '.input_search').send_keys(['맛집', Keys.ENTER]) # 맛집 검색
    time.sleep(0.5)

    # iframe 스위치
    driver.switch_to.frame('searchIframe')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    page_num = len(soup.select('.zRM9F > a')) - 1
        
    # 모든 페이지
    for p in range(page_num): 
        driver.switch_to.default_content()
        driver.switch_to.frame('searchIframe')

        place_lis = soup.select('.Ryr1F > ul > li') # 식당 리스트 가져오기
        print('number of places in page', p+1, ':', len(place_lis), ",", len(driver.find_elements(By.CSS_SELECTOR, '.tzwk0')))

        # 식당별
        for i in range(len(place_lis)): 

            # 식당 상세정보
            driver.find_elements(By.CSS_SELECTOR, '.tzwk0')[i].click()
            time.sleep(1)
            driver.switch_to.default_content()
            driver.switch_to.frame('entryIframe') # iframe 스위치
            soup1 = BeautifulSoup(driver.page_source, 'html.parser')

            # 식당 업종
            food_temp = soup1.select_one('.DJJvD').get_text().strip()

            # 식당 이름
            name = soup1.select_one('.Fc1rA').get_text().strip()

            # 식당 평점
            try:
                rating = soup1.select_one('.LXIwF').get_text().strip('별점').split('/')[0]
            except:
                rating = None # 평점이 없으면 Null 값으로 지정

            # 식당 주소
            addr = soup1.select_one('.LDgIH').get_text() # 식당 상세주소
            # if addr.split()[0] == '서울': # 만약 식당이 서울에 있다면
            gu_temp = addr.split()[1].strip() # 자치구 추출   
            # 리스트에 저장
            lis.append([gu_temp, food_temp, name, rating])
            print(gu_temp, food_temp, name, rating)
            # else: 
            #     continue # 주소지가 서울이 아니라면 스킵
                    
            driver.switch_to.default_content()
            driver.switch_to.frame('searchIframe')

        # 다음 페이지
        driver.find_elements(By.CSS_SELECTOR, '.eUTV2')[1].click() 
        time.sleep(1)
    
    # 드라이버 닫기
    driver.close()

 종로구
number of places in page 1 : 10 , 10
서초구 이탈리아음식 도셰프 4.38
마포구 와인 루나씨엘로 4.56
강남구 멕시코,남미음식 감성타코 가로수길점 4.42
종로구 일식당 오마카세 오사이초밥 대학로점 None
강남구 멕시코,남미음식 감성타코 강남역점 4.4
종로구 중식당 호랑이도삭면 종로점 None
중구 칼국수,만두 명동교자 본점 4.42
성동구 일식당 핫쵸 성수점 None
강남구 카페,디저트 미뉴트 빠삐용 None
강남구 퓨전음식 구구당 4.51
number of places in page 2 : 10 , 10
강남구 양식 을지다락 강남 4.53
중구 카페,디저트 적당 4.45
종로구 돈가스 정돈 대학로본점 4.48
서초구 스테이크,립 텍사스데브라질 센트럴시티점 4.31
종로구 베이커리 사월의물고기 4.58
종로구 딤섬,중식만두 티엔미미 서촌점 4.42
마포구 한식 올더플레이트 4.5
강남구 카페 노티드 청담 4.44
종로구 햄버거 다운타우너 안국 4.56
중구 일식당 쯔루하시 후게츠 명동점 4.49
number of places in page 3 : 10 , 10
강남구 멕시코,남미음식 더 타코부스 None
용산구 돼지고기구이 남영돈 None
종로구 카페 노티드 안국 4.45
마포구 분식 해피치즈스마일 연남점 4.65
마포구 육류,고기요리 육몽 홍대본점 4.73
중구 뷔페 서울신라호텔 더 파크뷰 4.55
마포구 냉면 을밀대 평양냉면 4.37
종로구 일식당 후라토식당 경복궁 본점 4.66
용산구 한식 능동미나리 None
종로구 한식 광화문미진 4.39
number of places in page 4 : 10 , 10
강남구 양식 테라스룸 4.3
성동구 곱창,막창,양 제일곱창 본점 4.45
강남구 낙지요리 뱃고동 4.28
마포구 피자 뉴오더클럽 연남 None
서대문구 카페 본지르르 None
서대문구 중식당 목란 None
마포구 베이커리 일쩜오플로어 None
강남구 이탈리아음식 마초쉐프 강남본점 4.71
중구 바(BAR) 롯

In [29]:
# 중복 제거
print('중복제거 전 :', len(lis))
food_place = []
[food_place.append(x) for x in lis if x not in food_place]
len('중복제거 후 :', food_place)

830

### 업종별 자치구별 평점 가져오기

In [8]:
# 모든 업종
lis = []

for food in food_lis: # 업종별
    print('///////////////////////////////////\n', food, '크롤링 시작') 
    for gu in gu_lis: # 자치구별
        
        try: 
            # 드라이버로 링크 열기
            options = webdriver.ChromeOptions()
            options.add_argument('--no-sandbox')
            options.add_argument('--disable-dev-shm-usage')
            options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")
            options.add_argument("headless")
            driver = webdriver.Chrome('chromedriver', options=options)

            driver.get('https://map.naver.com/')
            time.sleep(2)

            driver.find_element(By.CSS_SELECTOR, '.input_search').send_keys([gu, Keys.ENTER]) # 자치구 검색
            print('===================================\n', gu)
            time.sleep(1)
            driver.find_element(By.CSS_SELECTOR, '.button_clear').click() # 검색창 클리어
            driver.find_element(By.CSS_SELECTOR, '.input_search').send_keys([food, Keys.ENTER]) # 업종 검색
            time.sleep(0.5)

            # iframe 스위치
            driver.switch_to.frame('searchIframe')
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            page_num = len(soup.select('.zRM9F > a')) - 1
                
            # 모든 페이지
            for p in range(page_num): 
                driver.switch_to.default_content()
                driver.switch_to.frame('searchIframe')

                place_lis = soup.select('.Ryr1F > ul > li') # 식당 리스트 가져오기
                print('-----------------------------------\nnumber of places in page', p+1, ':', len(place_lis))

                # 식당별
                for i in range(len(place_lis)): 
                    try:
                        # 식당 상세정보
                        try:
                            driver.find_elements(By.CSS_SELECTOR, '.tzwk0')[i].click()
                        except:
                            driver.find_elements(By.CSS_SELECTOR, '.P7gyV')[i*2].click()
                        time.sleep(1)
                        driver.switch_to.default_content()
                        driver.switch_to.frame('entryIframe') # iframe 스위치
                        soup1 = BeautifulSoup(driver.page_source, 'html.parser')

                        # 식당 업종
                        try:
                            food_temp = soup1.select_one('.DJJvD').get_text().strip()
                        except:
                            food_temp = None

                        # 식당 이름
                        name = soup1.select_one('.Fc1rA').get_text().strip()

                        # 식당 평점
                        try:
                            rating = soup1.select_one('.LXIwF').get_text().strip('별점').split('/')[0]
                        except:
                            rating = None # 평점이 없으면 Null 값으로 지정

                        # 식당 주소
                        addr = soup1.select_one('.LDgIH').get_text() # 식당 상세주소
                        gu_temp = addr.split()[1].strip() # 자치구 추출   
                        
                        # 리스트에 저장
                        lis.append([gu_temp, addr, food_temp, name, rating])
                        print(gu_temp, food_temp, name, rating)

                        # 서치 iframe으로 스위치       
                        driver.switch_to.default_content()
                        driver.switch_to.frame('searchIframe')
                    except:
                        print(gu, food, (i+1) + "번째 식당 크롤링 실패")
                        continue

                # 다음 페이지
                driver.find_elements(By.CSS_SELECTOR, '.eUTV2')[1].click() 
                time.sleep(1)
            
            # 드라이버 닫기
            driver.close()
        
        except:
            print(gu, food, '크롤링 실패')
            continue

///////////////////////////////////
 치킨 크롤링 시작
 종로구
-----------------------------------
number of places in page 1 : 10
강북구 치킨,닭강정 교촌치킨 미아역점 4.3
강북구 치킨,닭강정 닭강정왕자떡볶이공주 None
종로구 치킨,닭강정 계열사 4.34
종로구 치킨,닭강정 효도치킨 광화문점 4.38
용산구 피자 보니스피자펍 4.47
중구 맥주,호프 만선호프 노가리체인본점 4.27
종로구 멕시코,남미음식 온더보더 광화문D타워점 4.38
종로구 맥주,호프 아트몬스터 익선동점 4.43
중구 맥주,호프 아트몬스터 을지로점 4.52
종로구 멕시코,남미음식 갓잇 안국점 None
-----------------------------------
number of places in page 2 : 10
마포구 치킨,닭강정 바른치킨 공덕파크자이점 4.36
성북구 요리주점 치치 성신여대점 4.58
종로구 햄버거 다운타우너 안국 4.56
중구 백숙,삼계탕 영양센터 본점 4.34
종로구 닭요리 와룡동닭매운탕 None
종로구 요리주점 내자살롱 4.51
서대문구 닭요리 용씨네 누룽지통닭구이 4.42
용산구 양식 마시앤바시 4.55
중구 맥주,호프 수표교호프 을지로본점 4.38
마포구 맥주,호프 크래프트한스 마포점 4.35
-----------------------------------
number of places in page 3 : 10
용산구 브런치 라페름 4.32
강북구 요리주점 도원결의 5
마포구 맥주,호프 금별맥주 공덕역점 None
종로구 요리주점 황금모이 4.55
강북구 치킨,닭강정 바른치킨 수유점 4.74
용산구 포장마차 해방집 4.92
중구 요리주점 다시열린하얀집 4.42
종로구 맥주,호프 이주당 4.39
강북구 맥주,호프 화수브루어리 서울강북점 4.56
종로구 치킨,닭강정 BBQ치킨 대학로점 None
-----------------------------------
number of

In [10]:
# 중복 제거
print('중복제거 전 :', len(lis))
food_place = []
[food_place.append(x) for x in lis if x not in food_place]
print('중복제거 후 :', len(food_place))

중복제거 전 : 14234
중복제거 후 : 8536


In [7]:
t_lis = []

In [29]:
# 실패한 업종
# t_lis = []
food = '일식'
t_gu_lis = ['양천구']

print('///////////////////////////////////\n', food, '크롤링 시작') 
for gu in t_gu_lis: # 자치구별
        
    try: 
        # 드라이버로 링크 열기
        options = webdriver.ChromeOptions()
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")
        options.add_argument("headless")
        driver = webdriver.Chrome('chromedriver', options=options)

        driver.get('https://map.naver.com/')
        time.sleep(2)

        driver.find_element(By.CSS_SELECTOR, '.input_search').send_keys([gu, Keys.ENTER]) # 자치구 검색
        print('===================================\n', gu)
        time.sleep(1)
        driver.find_element(By.CSS_SELECTOR, '.button_clear').click() # 검색창 클리어
        driver.find_element(By.CSS_SELECTOR, '.input_search').send_keys([food, Keys.ENTER]) # 업종 검색
        time.sleep(0.5)

        # iframe 스위치
        driver.switch_to.frame('searchIframe')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        page_num = len(soup.select('.zRM9F > a')) - 1
                
        # 모든 페이지
        for p in range(page_num): 
            driver.switch_to.default_content()
            driver.switch_to.frame('searchIframe')

            place_lis = soup.select('.Ryr1F > ul > li') # 식당 리스트 가져오기
            print('-----------------------------------\nnumber of places in page', p+1, '/', page_num, ':', len(place_lis))

            # 식당별
            for i in range(len(place_lis)): 
                try:
                    # 식당 상세정보
                    try:
                        driver.find_elements(By.CSS_SELECTOR, '.tzwk0')[i].click()
                    except:
                        driver.find_elements(By.CSS_SELECTOR, '.P7gyV')[i*2].click()
                    time.sleep(1)
                    driver.switch_to.default_content()
                    driver.switch_to.frame('entryIframe') # iframe 스위치
                    soup1 = BeautifulSoup(driver.page_source, 'html.parser')

                    # 식당 업종
                    try:
                        food_temp = soup1.select_one('.DJJvD').get_text().strip()
                    except:
                        food_temp = None
                     # 식당 이름
                    name = soup1.select_one('.Fc1rA').get_text().strip()

                    # 식당 평점
                    try:
                        rating = soup1.select_one('.LXIwF').get_text().strip('별점').split('/')[0]
                    except:
                        rating = None # 평점이 없으면 Null 값으로 지정

                    # 식당 주소
                    addr = soup1.select_one('.LDgIH').get_text() # 식당 상세주소
                    gu_temp = addr.split()[1].strip() # 자치구 추출   
                        
                    # 리스트에 저장
                    t_lis.append([gu_temp, addr, food_temp, name, rating])
                    print(i+1, ')', gu_temp, food_temp, name, rating)

                    # 서치 iframe으로 스위치       
                    driver.switch_to.default_content()
                    driver.switch_to.frame('searchIframe')
                except:
                    print(gu, food, (i+1) + "번째 식당 크롤링 실패")
                    continue

            # 다음 페이지
            driver.find_elements(By.CSS_SELECTOR, '.eUTV2')[1].click() 
            time.sleep(1)
            
        # 드라이버 닫기
        driver.close()
        
    except:
        print(gu, food, '크롤링 실패')
        continue

///////////////////////////////////
 일식 크롤링 시작
 양천구
-----------------------------------
number of places in page 1 / 6 : 10
1 ) 영등포구 이자카야 옥산 None
2 ) 금천구 샤브샤브 포미가 4.56
3 ) 마포구 이자카야 벽과담 4.5
4 ) 영등포구 일식당 문래동,멘 None
5 ) 영등포구 일식당 목화원 None
6 ) 강서구 생선회 안135 4.45
7 ) 양천구 일식당 삿뽀로 목동점 4.4
8 ) 양천구 일식당 갓포 쿠마카세 None
9 ) 마포구 일식당 모리타 None
10 ) 금천구 이자카야 텐마루 4.83
-----------------------------------
number of places in page 2 / 6 : 10
1 ) 강서구 일식당 가마메시야 울림 4.18
2 ) 영등포구 이자카야 미문의밤 None
3 ) 강서구 일식,초밥뷔페 더메이드뷔페 4.37
4 ) 마포구 초밥,롤 김영섭초밥 4.55
5 ) 관악구 초밥,롤 상무초밥 구로디지털점 None
6 ) 양천구 우동,소바 히노야마 4.47
7 ) 영등포구 이자카야 온도 4.92
8 ) 영등포구 이자카야 나니별관 4.46
9 ) 영등포구 이자카야 작은술집원지 4.93
10 ) 강서구 일식당 후라토식당 발산역점 4.56
-----------------------------------
number of places in page 3 / 6 : 10
1 ) 구로구 일식당 삿뽀로 디큐브점 4.25
2 ) 강서구 초밥,롤 이현우스시 4.69
3 ) 영등포구 일본식라면 하코야 문래역점 4.47
4 ) 영등포구 돈가스 문래동돈까스 4.49
5 ) 강서구 이자카야 호호방 4.68
6 ) 마포구 초밥,롤 오야스시 4.54
7 ) 마포구 일식당 온돈부리 상암점 4.51
8 ) 강서구 일식당 김희라 한국식왕돈까스 4.35
9 ) 강서구 샤브샤브 옥소반 마곡나루점 4.62
10 ) 영등포구 일식당 도쿄라

In [30]:
# 중복 제거
print('중복제거 전 :', len(t_lis))
t_food_place = []
[t_food_place.append(x) for x in t_lis if x not in t_food_place]
print('중복제거 후 :', len(t_food_place))

중복제거 전 : 3706
중복제거 후 : 2826


## 데이터 프레임으로 변환

In [11]:
# 데이터 프레임으로 변환
df = pd.DataFrame(food_place, columns=['gu', 'address', 'food', 'name', 'rating'])

In [46]:
# 데이터 프레임으로 변환
t_df = pd.DataFrame(t_food_place, columns=['gu', 'address', 'food', 'name', 'rating'])

In [31]:
# 데이터 프레임으로 변환
tt_df = pd.DataFrame(t_food_place, columns=['gu', 'address', 'food', 'name', 'rating'])

In [13]:
# 임시저장
df.to_csv('raw_rating_crawling.csv')

In [47]:
# 임시저장
t_df.to_csv('temp_raw_rating_crawling.csv')

In [79]:
# 임시저장
tt_df.to_csv('temp1_raw_rating_crawling.csv')

## 데이터 전처리

In [80]:
# 임시저장 데이터 불러오기
df = pd.read_csv('raw_rating_crawling.csv', index_col=0)
df1 = pd.read_csv('temp_raw_rating_crawling.csv', index_col=0)
df2 = pd.read_csv('temp1_raw_rating_crawling.csv', index_col=0)

In [81]:
# 데이터 합치기
df = pd.concat([df, df1, df2])
df

,gu,address,food,name,rating
0,강북구,서울 강북구 도봉로 142 1층,"치킨,닭강정",교촌치킨 미아역점,4.30
1,강북구,서울 강북구 도봉로76길 11 1층,"치킨,닭강정",닭강정왕자떡볶이공주,NaN
2,종로구,서울 종로구 백석동길 7,"치킨,닭강정",계열사,4.34
3,종로구,"서울 종로구 사직로8길 21-1 1, 2층","치킨,닭강정",효도치킨 광화문점,4.38
4,용산구,서울 용산구 신흥로3길 2,피자,보니스피자펍,4.47
...,...,...,...,...,...
2821,강서구,"서울 강서구 마곡중앙5로 81 2층 205호,206호,207호",샤브샤브,꽃마름 신방화점,4.31
2822,구로구,서울 구로구 경인로 661 신도림푸르지오상가 1층 117호(신도림동),이자카야,야보쿠시,NaN
2823,강서구,서울 강서구 마곡중앙6로 63 마곡테크노타워 109호,이자카야,토리아에즈,4.37
2824,양천구,서울 양천구 목동서로 170 홈플러스 1층 최우영스시 목동점,"초밥,롤",최우영스시 목동점,4.39


### 반복 및 결측치 제거

In [82]:
# 반복된 식당 제거
df.drop_duplicates(inplace=True)

,gu,address,food,name,rating
0,강북구,서울 강북구 도봉로 142 1층,"치킨,닭강정",교촌치킨 미아역점,4.30
1,강북구,서울 강북구 도봉로76길 11 1층,"치킨,닭강정",닭강정왕자떡볶이공주,NaN
2,종로구,서울 종로구 백석동길 7,"치킨,닭강정",계열사,4.34
3,종로구,"서울 종로구 사직로8길 21-1 1, 2층","치킨,닭강정",효도치킨 광화문점,4.38
4,용산구,서울 용산구 신흥로3길 2,피자,보니스피자펍,4.47
...,...,...,...,...,...
2819,마포구,서울 마포구 월드컵북로 352-16 1층,덮밥,시그니처키친,NaN
2821,강서구,"서울 강서구 마곡중앙5로 81 2층 205호,206호,207호",샤브샤브,꽃마름 신방화점,4.31
2823,강서구,서울 강서구 마곡중앙6로 63 마곡테크노타워 109호,이자카야,토리아에즈,4.37
2824,양천구,서울 양천구 목동서로 170 홈플러스 1층 최우영스시 목동점,"초밥,롤",최우영스시 목동점,4.39


In [83]:
# 결측치(평점 없는 식당) 제거
df.dropna(inplace=True)
df

,gu,address,food,name,rating
0,강북구,서울 강북구 도봉로 142 1층,"치킨,닭강정",교촌치킨 미아역점,4.30
2,종로구,서울 종로구 백석동길 7,"치킨,닭강정",계열사,4.34
3,종로구,"서울 종로구 사직로8길 21-1 1, 2층","치킨,닭강정",효도치킨 광화문점,4.38
4,용산구,서울 용산구 신흥로3길 2,피자,보니스피자펍,4.47
5,중구,서울 중구 을지로13길 19,"맥주,호프",만선호프 노가리체인본점,4.27
...,...,...,...,...,...
2817,강서구,서울 강서구 강서로 299 1층 오모우스시,일식당,오모우스시,4.35
2818,부천시,경기 부천시 옥길로 110-11 4층,샤브샤브,채선당 자연한가득 부천옥길점,4.49
2821,강서구,"서울 강서구 마곡중앙5로 81 2층 205호,206호,207호",샤브샤브,꽃마름 신방화점,4.31
2823,강서구,서울 강서구 마곡중앙6로 63 마곡테크노타워 109호,이자카야,토리아에즈,4.37


### 서울시 자치구 데이터만 불러오기

In [84]:
print(df.gu.unique())
print(df.gu.nunique())

['강북구' '종로구' '용산구' '중구' '마포구' '성북구' '서대문구' '성동구' '도봉구' '동대문구' '강남구' '송파구'
 '광진구' '서초구' '중랑구' '관악구' '강서구' '영등포구' '광명시' '금천구' '구로구' '양천구' '동작구' '안양시'
 '과천시' '의왕시' '성남시' '강동구' '용인시' '하남시' '구리시' '남양주시' '노원구' '의정부시' '은평구' '고양시'
 '부천시' '부평구' '춘천시' '수원시' '오산시' '화성시' '광주시' '유성구' '순천시' '포항시' '동구' '수성구'
 '안산시' '파주시' '김포시' '청주시' '원주시' '양주시' '공주시' '아산시' '서구' '달서구' '포천시' '남구'
 '무안군' '군산시' '북구' '이천시' '서산시' '시흥시' '홍성군' '기장군' '연수구' '목포시' '부여군' '동두천시'
 '논산시' '당진시' '계양구' '영광군' '천안시' '미추홀구' '남동구' '전주시' '군포시']
81


In [85]:
# 서울시 자치구가 아닌 데이터 제거
df = df[(df.gu != '구리시') & (df.gu != '의정부시') & (df.gu != '남양주시') & (df.gu != '시흥시') & (df.gu != '광명시') & (df.gu != '안양시') & (df.gu != '하남시') & (df.gu != '성남시') 
            & (df.gu != '고양시') & (df.gu != '과천시') & (df.gu != '의왕시') & (df.gu != '용인시') & (df.gu != '부천시') & (df.gu != '부평구') & (df.gu != '춘천시') & (df.gu != '수원시') 
            & (df.gu != '오산시') & (df.gu != '화성시') & (df.gu != '광주시') & (df.gu != '유성구') & (df.gu != '순천시') & (df.gu != '포항시') & (df.gu != '동구') & (df.gu != '수성구') 
            & (df.gu != '안산시') & (df.gu != '파주시') & (df.gu != '김포시') & (df.gu != '청주시') & (df.gu != '원주시') & (df.gu != '양주시') & (df.gu != '공주시') & (df.gu != '아산시')
             & (df.gu != '서구') & (df.gu != '달서구') & (df.gu != '포천시') & (df.gu != '남구') & (df.gu != '무안군') & (df.gu != '군산시') & (df.gu != '북구') & (df.gu != '이천시')
              & (df.gu != '서산시') & (df.gu != '홍성군') & (df.gu != '기장군') & (df.gu != '연수구') & (df.gu != '목포시') & (df.gu != '부여군') & (df.gu != '동두천시') & (df.gu != '논산시')
               & (df.gu != '당진시') & (df.gu != '계양구') & (df.gu != '영광군') & (df.gu != '천안시') & (df.gu != '미추홀구') & (df.gu != '남동구') & (df.gu != '전주시') & (df.gu != '군포시')]

In [86]:
print(df.gu.unique())
print(df.gu.nunique())

['강북구' '종로구' '용산구' '중구' '마포구' '성북구' '서대문구' '성동구' '도봉구' '동대문구' '강남구' '송파구'
 '광진구' '서초구' '중랑구' '관악구' '강서구' '영등포구' '금천구' '구로구' '양천구' '동작구' '강동구' '노원구'
 '은평구']
25


### 데이터 타입 수정

In [88]:
df.dtypes

gu          object
address     object
food        object
name        object
rating     float64
dtype: object

In [89]:
# 평점을 float 데잍터 타입으로 변환
df.rating = df.rating.astype('float')

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_9812\2914775465.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rating = df.rating.astype('float')


In [90]:
df.dtypes

gu          object
address     object
food        object
name        object
rating     float64
dtype: object

### 요식업종 통일

In [91]:
# 인덱스 리셋
df.set_index([pd.Index(range(0, df.gu.count()))], inplace=True)

In [92]:
print(df.food.unique())
print(df.food.nunique())

['치킨,닭강정' '피자' '맥주,호프' '멕시코,남미음식' '요리주점' '햄버거' '백숙,삼계탕' '닭요리' '양식' '브런치'
 '포장마차' '인도음식' '일식당' '돈가스' '이자카야' '스테이크,립' '족발,보쌈' '육류,고기요리' '돼지고기구이'
 '닭갈비' '중식당' '이탈리아음식' '우동,소바' '인근주민' '순대,순댓국' '오리요리' '다이어트,샐러드' '칼국수,만두'
 '한식' '냉면' '곰탕,설렁탕' '국수' '전,빈대떡' '장어,먹장어요리' '생선구이' '해물,생선요리' '감자탕'
 '곱창,막창,양' '아귀찜,해물찜' '닭볶음탕' '한정식' '오징어요리' '주꾸미요리' '소고기구이' '이북음식' '게요리'
 '닭발' '조개요리' '해장국' '찌개,전골' '생선회' '기사식당' '매운탕,해물탕' '백반,가정식' '찜닭' '막국수'
 '안경할머니곱창' '갈비탕' '낙지요리' '두부요리' '정육식당' '일도씨닭갈비' '국밥' '굴요리' '추어탕' '대구형제막창'
 '쌈밥' '복어요리' '문어부인삼교비' '문래돼지불백' '한식뷔페' '강화통통생고기' '빙수' '보리밥' '포장용상자제조'
 '샌드위치' '종합분식' '바닷가재요리' '초밥,롤' '전복요리' '대게나라' '달빛오징어광어' '킹크랩요리' '떡볶이' '분식'
 '만두' '김밥' '라면' '토스트' '호떡' '카페,디저트' '도시락,컵밥' '케이크전문' '딤섬,중식만두' '양꼬치' '야식'
 '스파게티,파스타전문' '닭장수후라이드' '태국음식' '아시아음식' '베트남음식' '뷔페' '패밀리레스토랑' '푸드코트'
 '퓨전음식' '프랑스음식' '터키음식' '스페인음식' '서오릉피자' '핫도그' '스파게티스토리' '독일음식' '그리스음식'
 '족황상제' '샤브샤브' '101번지남산돈까스' '미스터빠삭' '일본식라면' '덮밥' '카레' '일식,초밥뷔페' '일식튀김,꼬치'
 '오므라이스' '바(BAR)' '달떡볶이' '고고즉석떡볶이' '가공식품' '33떡볶이' '카페' '과일,주스전문점'
 '갈비배달

In [93]:
# 필요없는 식당 업종 제거
df = df[(df.food != '북카페') & (df.food != '꽃집,꽃배달') & (df.food != '인근주민') & (df.food != '스터디카페') & (df.food != '라이브카페') & (df.food != '플라워카페') & (df.food != '테마카페')
         & (df.food != '갤러리,화랑') & (df.food != '갤러리카페') & (df.food != '포장용상자제조') & (df.food != '가공식품') & (df.food != '목욕탕,사우나')]

In [94]:
# 음식업종 카테고리 정리
for i in df.index:
    if df.loc[i, 'food'] == '이탈리아음식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '다이어트,샐러드': df.loc[i, 'food'] = '카페/디저트'          ####
    elif df.loc[i, 'food'] == '독일음식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '백숙,삼계탕': df.loc[i, 'food'] = '찜탕'
    elif df.loc[i, 'food'] == '포장마차': df.loc[i, 'food'] = '야식'
    elif df.loc[i, 'food'] == '그리스음식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '족황상제': df.loc[i, 'food'] = '족발/보쌈'
    elif df.loc[i, 'food'] == '101번지남산돈까스': df.loc[i, 'food'] = '돈까스/일식'
    elif df.loc[i, 'food'] == '미스터빠삭': df.loc[i, 'food'] = '돈까스/일식'
    elif df.loc[i, 'food'] == '일식,초밥뷔페': df.loc[i, 'food'] = '돈까스/일식'
    elif df.loc[i, 'food'] == '오므라이스': df.loc[i, 'food'] = '돈까스/일식'
    elif df.loc[i, 'food'] == '달떡볶이': df.loc[i, 'food'] = '분식'
    elif df.loc[i, 'food'] == '고고즉석떡볶이': df.loc[i, 'food'] = '분식'
    elif df.loc[i, 'food'] == '33떡볶이': df.loc[i, 'food'] = '분식'
    elif df.loc[i, 'food'] == '과일,주스전문점': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '갈비배달도시락스트릿테이블': df.loc[i, 'food'] = '도시락'
    elif df.loc[i, 'food'] == '신룽푸마라탕': df.loc[i, 'food'] = '중식'
    elif df.loc[i, 'food'] == '아이스크림': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '한방카페': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '스마일명품찹쌀꽈배기': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '찐빵': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '차': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '호두과자': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '블루보틀': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '강화통통생고기': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '빙수': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '보리밥': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '전복요리': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '대게나라': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '달빛오징어광어': df.loc[i, 'food'] = '회'
    elif df.loc[i, 'food'] == '킹크랩요리': df.loc[i, 'food'] = '회'
    elif df.loc[i, 'food'] == '라면': df.loc[i, 'food'] = '분식'
    elif df.loc[i, 'food'] == '토스트': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '호떡': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '도시락,컵밥': df.loc[i, 'food'] = '도시락'
    elif df.loc[i, 'food'] == '닭장수후라이드': df.loc[i, 'food'] = '치킨'
    elif df.loc[i, 'food'] == '푸드코트': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '프랑스음식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '터키음식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '서오릉피자': df.loc[i, 'food'] = '피자'
    elif df.loc[i, 'food'] == '핫도그': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '스파게티스토리': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '전,빈대떡': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '생선구이': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '이북음식': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '게요리': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '기사식당': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '매운탕,해물탕': df.loc[i, 'food'] = '찜탕'
    elif df.loc[i, 'food'] == '찜닭': df.loc[i, 'food'] = '찜탕'
    elif df.loc[i, 'food'] == '안경할머니곱창': df.loc[i, 'food'] = '야식'
    elif df.loc[i, 'food'] == '갈비탕': df.loc[i, 'food'] = '찜탕'
    elif df.loc[i, 'food'] == '정육식당': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '일도씨닭갈비': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '국밥': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '굴요리': df.loc[i, 'food'] = '회'
    elif df.loc[i, 'food'] == '추어탕': df.loc[i, 'food'] = '찜탕'
    elif df.loc[i, 'food'] == '대구형제막창': df.loc[i, 'food'] = '야식'
    elif df.loc[i, 'food'] == '쌈밥': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '복어요리': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '문래돼지불백': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '한식뷔페': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '와인': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '멕시코,남미음식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '칼국수,만두': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '양식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '카페,디저트': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '돈가스': df.loc[i, 'food'] = '돈까스/일식'
    elif df.loc[i, 'food'] == '스테이크,립': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '베이커리': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '딤섬,중식만두': df.loc[i, 'food'] = '중식'
    elif df.loc[i, 'food'] == '카페': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '햄버거': df.loc[i, 'food'] = '패스트푸드'
    elif df.loc[i, 'food'] == '일식당': df.loc[i, 'food'] = '돈까스/일식'
    elif df.loc[i, 'food'] == '뷔페': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '냉면': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '곱창,막창,양': df.loc[i, 'food'] = '야식'           
    elif df.loc[i, 'food'] == '낙지요리': df.loc[i, 'food'] = '한식'                
    elif df.loc[i, 'food'] == '바(BAR)': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '중식당': df.loc[i, 'food'] = '중식'
    elif df.loc[i, 'food'] == '일본식라면': df.loc[i, 'food'] = '돈까스/일식'
    elif df.loc[i, 'food'] == '종합분식': df.loc[i, 'food'] = '분식'
    elif df.loc[i, 'food'] == '맥주,호프': df.loc[i, 'food'] = '야식'
    elif df.loc[i, 'food'] == '감자탕': df.loc[i, 'food'] = '찜탕'
    elif df.loc[i, 'food'] == '스페인음식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '만두': df.loc[i, 'food'] = '야식'                    
    elif df.loc[i, 'food'] == '순대,순댓국': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '돼지고기구이': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '베트남음식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '떡볶이': df.loc[i, 'food'] = '분식'
    elif df.loc[i, 'food'] == '우동,소바': df.loc[i, 'food'] = '돈까스/일식'
    elif df.loc[i, 'food'] == '곰탕,설렁탕': df.loc[i, 'food'] = '찜탕'
    elif df.loc[i, 'food'] == '닭발': df.loc[i, 'food'] = '야식'
    elif df.loc[i, 'food'] == '일식튀김,꼬치': df.loc[i, 'food'] = '돈까스/일식'
    elif df.loc[i, 'food'] == '도넛': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '막국수': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '장어,먹장어요리': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '브런치': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '덮밥': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '인도음식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '초밥,롤': df.loc[i, 'food'] = '돈까스/일식'
    elif df.loc[i, 'food'] == '주꾸미요리': df.loc[i, 'food'] = '한식'                          
    elif df.loc[i, 'food'] == '백반,가정식': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '국수': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '스파게티,파스타전문': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '닭요리': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '전통,민속주점': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '조개요리': df.loc[i, 'food'] = '야식'
    elif df.loc[i, 'food'] == '패밀리레스토랑': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '해물,생선요리': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '태국음식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '족발,보쌈': df.loc[i, 'food'] = '족발/보쌈'
    elif df.loc[i, 'food'] == '샌드위치': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '아시아음식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '찌개,전골': df.loc[i, 'food'] = '찜탕'
    elif df.loc[i, 'food'] == '샤브샤브': df.loc[i, 'food'] = '찜탕'
    elif df.loc[i, 'food'] == '아귀찜,해물찜': df.loc[i, 'food'] = '찜탕'
    elif df.loc[i, 'food'] == '김밥': df.loc[i, 'food'] = '분식'
    elif df.loc[i, 'food'] == '소고기구이': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '치킨,닭강정': df.loc[i, 'food'] = '치킨'
    elif df.loc[i, 'food'] == '크레페': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '이자카야': df.loc[i, 'food'] = '돈까스/일식'
    elif df.loc[i, 'food'] == '베이글': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '닭갈비': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '카레': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '케이크전문': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '두부요리': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '오징어요리': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '생선회': df.loc[i, 'food'] = '회'
    elif df.loc[i, 'food'] == '문어부인삼교비': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '오리요리': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '한정식': df.loc[i, 'food'] = '한식'
    elif df.loc[i, 'food'] == '와플': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '테이크아웃커피': df.loc[i, 'food'] = '카페/디저트'
    elif df.loc[i, 'food'] == '닭볶음탕': df.loc[i, 'food'] = '찜탕'
    elif df.loc[i, 'food'] == '바닷가재요리': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '해산물뷔페': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '해장국': df.loc[i, 'food'] = '한식'                             
    elif df.loc[i, 'food'] == '양꼬치': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '퓨전음식': df.loc[i, 'food'] = '아시안/양식'
    elif df.loc[i, 'food'] == '육류,고기요리': df.loc[i, 'food'] = '한식'                             
    elif df.loc[i, 'food'] == '요리주점': df.loc[i, 'food'] = '아시안/양식'    
    elif df.loc[i, 'food'] == '밀도': df.loc[i, 'food'] = '카페/디저트'                         

In [95]:
df.food.unique()

array(['치킨', '피자', '야식', '아시안/양식', '패스트푸드', '찜탕', '한식', '돈까스/일식', '족발/보쌈',
       '중식', '카페/디저트', '회', '분식', '도시락'], dtype=object)

### 데이터 용량 확인 및 인덱스 리셋

In [97]:
# 데이터 용량 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10510 entries, 0 to 10509
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   gu       10510 non-null  object 
 1   address  10510 non-null  object 
 2   food     10510 non-null  object 
 3   name     10510 non-null  object 
 4   rating   10510 non-null  float64
dtypes: float64(1), object(4)
memory usage: 410.7+ KB


In [98]:
# 인덱스 리셋
df.set_index([pd.Index(range(0, df.gu.count()))], inplace=True)

### API로 위도 경도 불러오기

In [99]:
# 카카오 API KEY 불러오기
with open('D:/박상춘/프로젝트/raw data/카톡_api_key.txt', "r") as f:
    api_key = f.read()

In [100]:
# 위도, 경도 추출
lat_lis, lng_lis = [], []

for i in tqdm(df['address']):
    addr = f'{(i)}'
    quote(addr)
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(addr)}'
    result = requests.get(url, headers = {'Authorization': f'KakaoAK {api_key}'}).json()
    if not result['documents']:
        print(f"No result found for {addr}") # 위도 경도를 못 찾을 경우, None 값을 넣고 프린트
        lat_lis.append(None)
        lng_lis.append(None)
    else:
        lng = float(result['documents'][0]['x'])
        lat = float(result['documents'][0]['y'])
        lat_lis.append(lat)
        lng_lis.append(lng)

  0%|          | 21/10510 [00:01<10:30, 16.64it/s]

No result found for 서울 용산구 대사관로11길 47 (대사관로 11길 49) 지하 1층


  1%|          | 113/10510 [00:06<10:10, 17.03it/s]

No result found for 서울 종로구 삼일대로26길 10 1F 'VoVo pizza'


  1%|▏         | 149/10510 [00:09<11:51, 14.56it/s]

...

100%|█████████▉| 10472/10510 [12:05<00:02, 16.16it/s]

No result found for 서울 강서구 마곡중앙로 59-17 118~122호 1층


100%|█████████▉| 10500/10510 [12:07<00:00, 13.55it/s]

No result found for 서울 금천구 디지털로10길 9 가산동 6층 612호 현대시티아울렛 가산점


100%|██████████| 10510/10510 [12:08<00:00, 14.43it/s]


In [101]:
# 데이터프레임에 위도, 경도 값 추가
df['lat'] = lat_lis
df['lng'] = lng_lis

In [102]:
# None 값 확인
df.isnull().sum()

gu           0
address      0
food         0
name         0
rating       0
lat        113
lng        113
dtype: int64

In [103]:
# None 값으로 입력된 데이터 제거
df.dropna(inplace=True)

In [108]:
# 인덱스 리셋
df.set_index([pd.Index(range(0, df.gu.count()))], inplace=True)

## csv 파일로 변환

In [109]:
df.to_csv('rating_crawling.csv')

## 데이터 분석 및 시각화

In [110]:
df.groupby(['gu', 'food']).agg('mean', 'count')

rating        lat         lng
gu  food                                   
강남구 도시락     4.424615  37.507254  127.043569
    돈까스/일식  4.483534  37.510724  127.040378
    분식      4.359661  37.506740  127.041308
    아시안/양식  4.503675  37.513159  127.038638
    야식      4.483913  37.508099  127.033538
...              ...        ...         ...
중랑구 카페/디저트  4.290000  37.590632  127.093172
    패스트푸드   4.434118  37.598372  127.086945
    피자      4.438000  37.597381  127.089557
    한식      4.385385  37.596132  127.087722
    회       4.446000  37.601366  127.084334

[350 rows x 3 columns]

In [70]:
geo_data = json.load(open('D:/박상춘/프로젝트/raw data/서울시 행정 데이터/seoul_municipalities_geo_simple.json', encoding='utf-8'))

In [ ]:
map = folium.Map(location=[37.541, 126.986], zoom_start=12)
marker_cluster = MarkerCluster().add_to(map)
GeoJson(geo_data).add_to(map)
for i in df.index[:]:
    folium.Marker(location = [df.lat[i], df.lng[i]],
                popup = folium.Popup((f'{df.name[i]}({df.food[i]}): {df.rating[i]}'), max_width=200), 
                tooltip=df.rating[i],
                icon=folium.Icon(color = 'orange', icon = 'glyphicon glyphicon-star')
                ).add_to(marker_cluster)
map